In [1]:
!pwd

/nfs/Workspace/CardiacSeg


In [2]:
%cd /nfs/Workspace/CardiacSeg

/nfs/Workspace/CardiacSeg


In [8]:
import os

import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

# Quantitative Evaluations

In [67]:
class EvalResult:
    def __init__(self, csv_pth, labels):
        self.pid_col = ['patientId']
        self.dc_cols = [f'dice{lab}' for lab in labels]
        self.hd_cols = [f'hd95{lab}' for lab in labels]
        self.csv_pth = csv_pth
        self.df = pd.read_csv(csv_pth)
    
    def cal_avg(self):
        self.avg_dc_df = self.cal_avg_metric_df(self.dc_cols)
        self.avg_dc =  self.avg_dc_df['avg'].mean()
        self.avg_hd_df = self.cal_avg_metric_df(self.hd_cols)
        self.avg_hd =  self.avg_hd_df['avg'].mean()
    
    def cal_avg_metric_df(self, met_cols):
        avg_met_by_pid = self.df[met_cols].T.mean().to_frame(name='avg')
        return pd.concat(
              [self.df[self.pid_col + met_cols], avg_met_by_pid], 
              axis=1, 
              join='inner'
            ).reset_index(drop=True)

    
class SegTHOREvalResult(EvalResult):
    def __init__(self, csv_pth):
        self.labels = ['']
        super().__init__(csv_pth, self.labels)
        self.cal_avg()
        
class MMWHSEvalResult(EvalResult):
    def __init__(self, csv_pth):
        self.labels = ['LV', 'RV', 'LA', 'RA', 'MLV', 'AA', 'PA']
        super().__init__(csv_pth, self.labels)
        self.cal_avg()
        

def get_csv_dict(root_dir, model_names, exp_names, data_name):
    csv_file_name = 'best_model_eval.csv'
    csv_dict = {}
    for m in model_names:
         csv_dict[m] = [os.path.join(root_dir, m, data_name, 'evals', exp_name, csv_file_name) for exp_name in exp_names]
    return csv_dict        
        

def get_eval_dict(csv_dict, evalResult):
    eval_dict = {}
    for model_name, csv_pths in csv_dict.items():
        eval_dict[model_name] =  [evalResult(csv_pth) for csv_pth in csv_pths]
    return eval_dict


def get_avg_fold_df(eval_dict):
    models = []
    avg_fd_dcs = []
    avg_fd_hds = []
    for m, fold_dfs in eval_dict.items():
        avg_dc = np.mean([f_df.avg_dc for f_df in fold_dfs])
        avg_hd = np.mean([f_df.avg_hd for f_df in fold_dfs])

        avg_fd_dcs.append(avg_dc)
        avg_fd_hds.append(avg_hd)
        models.append(m)

    avg_fd_df = pd.DataFrame({
        'model': models,
        'dice': avg_fd_dcs,
        'hd95': avg_fd_hds
    })
    return avg_fd_df

In [69]:
root_dir = '/nfs/Workspace/CardiacSeg'
model_names = ['UNet3D', 'AttentionUNet', 'CoTr', 'UNETR', 'SwinUNETR', 'UNETCNX'] 
exp_names = [f'exp_1_fold_{i}' for i in range(0, 5)]

segthor_csv_dict = get_csv_dict(root_dir, model_names, exp_names, data_name='SegTHOR')
mmwhs_csv_dict = get_csv_dict(root_dir, model_names, exp_names, data_name='MMWHS')

s_eval_dict = get_eval_dict(segthor_csv_dict, SegTHOREvalResult)
m_eval_dict = get_eval_dict(mmwhs_csv_dict, MMWHSEvalResult)

s_avg_fd_df = get_avg_fold_df(s_eval_dict)
m_avg_fd_df = get_avg_fold_df(m_eval_dict)

## SegTHOR

In [70]:
s_avg_fd_df

,model,dice,hd95
0,UNet3D,0.687799,27.270456
1,AttentionUNet,0.850992,21.240324
2,CoTr,0.949371,4.134993
3,UNETR,0.947963,7.292262
4,SwinUNETR,0.953803,3.709072
5,UNETCNX,0.952352,5.266610


## MM_WHS

In [71]:
m_avg_fd_df

,model,dice,hd95
0,UNet3D,0.739152,51.380736
1,AttentionUNet,0.842547,40.543823
2,CoTr,0.913951,12.283495
3,UNETR,0.847109,35.867616
4,SwinUNETR,0.905947,26.422505
5,UNETCNX,0.909063,21.345425
